In [ ]:
import warnings
import typing as tp

import numpy as np
import scipy.linalg as sla

In [4]:
def compute_alpha(inverse_matrix: np.array, column: np.array, column_no: int) -> float:
    return inverse_matrix[column_no] @ column

def compute_inverse_step(previous_inverse_matrix: np.array, column: np.array, column_no: int) -> np.array:
    d = previous_inverse_matrix @ column
    d_k = d[column_no]
    d[column_no] = -1
    d /= -d_k
    
    D = np.eye(previous_inverse_matrix.shape[0])
    D[:, column_no] = d
    
    return D @ previous_inverse_matrix

def inverse(matrix: np.array) -> np.array:
    current_inverse_matrix = np.eye(matrix.shape[0])
    iteration = 0
    
    indexes_set = set(range(matrix.shape[0]))
    indexes_order = [0] * matrix.shape[0]
    
    while iteration < matrix.shape[0]:
        is_singular = True
        for current_column_no in indexes_set:
            current_column = matrix[: ,current_column_no]
            alpha = compute_alpha(current_inverse_matrix, current_column, iteration)
            if not np.isclose(alpha, 0.0, rtol=1e-11):
                is_singular = False
                indexes_set.remove(current_column_no)
                indexes_order[current_column_no] = iteration
                break
        if is_singular:
            warnings.warn("Singular matrix", sla.LinAlgWarning)
            return None    
        current_inverse_matrix = compute_inverse_step(current_inverse_matrix, current_column, iteration)
        iteration += 1
    return current_inverse_matrix[np.array(indexes_order), :]

In [5]:
def test(matrix: np.array, inverse_matrix: np.array, eps = 1e-5) -> None:
    if inverse_matrix is not None:
        print(matrix @ inverse_matrix)
        assert sla.norm(matrix @ inverse_matrix - np.eye(matrix.shape[0])) < eps

In [6]:
matrices = [
    np.array([[0, 0, 0, 1], [0, 0, 1, 0], [0, 1, 0, 0], [1, 0, 0, 0]]), 
    
    np.array([[0, 0, 0, -5], [0, 0, 1, 6], [2, 4, 0, 0], [-1, 3, 0, 0]]),
    np.array([[0, 0, 4, 0], [0, 0, 1, 6], [1, 4, 0, 0], [-1, 3, 0, 0]]),
    
    np.array([[0, 0, 8, 3], [0, 0, 1, 6], [2, 4, 0, 0], [0, 3, 0, 0]]),
    np.array([[0, 0, 0, -5], [0, 0, 1, 6], [0, 4, 0, 0], [-1, 0, 0, 0]]),
    np.array([[0, 0, 0, -5], [0, 0, 2, 6], [0, 4, 1, 3], [-1, 2, 1, 6]]),
    
    np.array([[0, 0, 0, 1], [0, 0, 1, 6], [0, 2, 1, -3], [1, 2, 1, 6]]),
    np.array([[0, 0, 0, 1], [1, 0, 2, 6], [0, 2, 1, -3], [1, 2, 1, 6]]),
    np.array([[0, 0, 0, 1], [1, 0, 2, -1], [0, 2, 1, -3], [0, 2, 1, 6]]),
    
    np.array([[0, 1, 1, -1], [1, 0, 2, 6], [0, 2, 1, -3], [1, 2, 1, -1]]),
    np.array([[0, 1, 1, -1], [1, 0, 1, 3], [0, 1, 1, -3], [1, 2, 1, -1]]),
    np.array([[0, 1, 0, -1], [0, 0, 2, 6], [0, 2, 1, -3], [1, -2, 1, 1]]),

    np.array([[-1, 1, 0, -1], [0, 0, 2, 6], [1, 2, 1, -3], [1, -2, 1, 1]]),
    np.array([[0, 1, 0, 0], [0, 0, 2, 4], [1, 2, 1, -3], [1, -2, 1, 1]]),
    np.array([[-1, -1, -1, -1], [0, 0, 2, 6], [1, 2, 0, 0], [1, -2, 1, 1]]),
]


epses = [1, 0.1, 1e-6, 1e-9, 1e-15]
B = lambda eps: np.array([[2, 8, -1, 4, 5, 6], 
                          [1, -9, 2, -3, 1, -2], 
                          [3, -1, 1, 1 + eps, 6, 4],
                          [0, 1, 0, 1, 0, 2], 
                          [1, 2, -1, 4, 2, 3],
                          [-3, 2, 1, 0, 0, 0]])

In [9]:
inverse(matrices[9])

array([[ 4. , -1. , -4. ,  2. ],
       [-5. ,  1. ,  4. , -1. ],
       [ 4. , -0.5, -2.5,  0.5],
       [-2. ,  0.5,  1.5, -0.5]])